In [1]:
import json
import requests
from requests.auth import HTTPBasicAuth
import pandas as pd
from plotly import utils

In [2]:
username = 'removegrids'    #however in this example the account will be 'yankev'
api_key = 'jl9towemhr'

auth = HTTPBasicAuth(username, api_key)
headers = {'Plotly-Client-Platform': 'python'}
page_size = 500

In [ ]:
# Create a grid for testing

In [ ]:
# this method uses pre-defined functions in the Plotly.py library

from plotly.grid_objs import Column, Grid
import plotly.plotly as py

column_1 = Column([1, 2, 3], 'column 1')
grid = Grid([column_1])

url = py.grid_ops.upload(grid, 
                         'grid example9999',      # name of the grid in your plotly account
                         world_readable=True, # public or private
                         auto_open=False)      # open the grid in the browser

In [12]:
# alternatively you can make your own requests

payload = {'filename': 'testgrid',
          "data": json.dumps({"cols": {"first": {"data": [1,2,3], "order": 0}}}, cls=utils.PlotlyJSONEncoder),
          "world_readable": True}
response = requests.post('https://api.plot.ly/v2/grids/', auth=auth, headers=headers, data=payload)


In [ ]:
# get the fid of the grid that was just created

In [36]:
grid = json.loads(response.content)['file']['fid'].encode('utf8')
grid

'removegrids:0'

In [ ]:
# Delete the grid

In [42]:
response = requests.delete('https://api.plot.ly/v2/grids/{}'.format(grid), auth=auth, headers=headers)
print response.status_code
print response.content
# response = requests.delete('https://api.plot.ly/v2/grids/{}/permanent_delete'.format(grid), auth=auth, headers=headers)

204



In [ ]:
# Permanently delete the grid

In [44]:
response = requests.delete('https://api.plot.ly/v2/grids/{}/permanent_delete'.format(grid), auth=auth, headers=headers)
print response.status_code
print response.content

204



### Below are some functions that you could use in order to get all the files

In [48]:
def get_files(file):
    '''
    This function will the name, fid, and path of all the files in your
    plotly repository with the file type you choose
    -----
    Arguments
    file: the type of file we want to retrieve and takes on values ['plot', 'fold', 'grid']
    
    Returns: A list of lists, containing the information required
    
    '''
    
    url = 'https://api.plot.ly/v2/folders/all?user='+username+'&filetype='+file+'&page_size='+str(page_size)
    response = requests.get(url, auth=auth, headers=headers)
    page = json.loads(response.content)
    temp = []
    for i in range(page['children']['count']):
        path = requests.get('https://api.plot.ly/v2/files/{}/path'.format(page['children']['results'][i]['fid']), auth=auth, headers=headers)
        #print('Filename: {}, fid: {}, path: {}'.format(page['children']['results'][i]['filename'],page['children']['results'][i]['fid'],str.split(path.content,":")[1][1:-2]))
        temp.append([page['children']['results'][i]['filename'],page['children']['results'][i]['fid'],str.split(path.content,":")[1][1:-2]])
    return temp 

grids = get_files('grid')

In [49]:
def files2df(data):
    '''
    This function will turn your data into a dataframe
    -----
    Arguments
    data: Output of get_files call
    
    Returns: a data frame containing information in "data"
    '''
    return pd.DataFrame(data,columns=['name','fid','path'])

df_grids = files2df(grids)

In [50]:
df_grids

,name,fid,path
